In [ ]:
!pip install transformers


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy --upgrade

  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [2]:
import pandas as pd

# Upload train.csv and test.csv
# from google.colab import files
# uploaded = files.upload()

train_df = pd.read_csv("/content/drive/MyDrive/Web_Mining_DistilBert/balanced_train_data_updated.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Web_Mining_DistilBert/balanced_test_data_updated.csv")

# Combine columns if needed
train_df['text'] = train_df['Summary'].fillna('') + " " + train_df['Text'].fillna('')
test_df['text'] = test_df['Summary'].fillna('') + " " + test_df['Text'].fillna('')

# Map labels to integers if not already encoded
train_df['label'] = train_df['Score']
test_df['label'] = test_df['Score']

In [3]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def encode(texts, labels, max_len=128):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len, return_tensors='pt')
    return encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

class BERTDataset(Dataset):
    def __init__(self, texts, labels):
        self.input_ids, self.attn_mask, self.labels = encode(texts, labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attn_mask[idx],
            'labels': self.labels[idx]
        }

train_data = BERTDataset(train_df['text'].tolist(), train_df['label'].tolist())
test_data = BERTDataset(test_df['text'].tolist(), test_df['label'].tolist())

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16)


In [5]:
from transformers import DistilBertForSequenceClassification

num_labels = len(set(train_df['label']))  # 3 for sentiment (0, 1, 2)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [6]:
import torch.nn as nn
from torch.optim import AdamW
from tqdm import tqdm
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Optimizer and loss
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Config
EPOCHS = 10
CHECKPOINT_DIR = "/content/drive/MyDrive/Web_Mining_DistilBert/checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
latest_path = os.path.join(CHECKPOINT_DIR, "latest.pt")
best_path = os.path.join(CHECKPOINT_DIR, "best.pt")

# Resume if latest checkpoint exists
start_epoch = 0
best_loss = float('inf')

if os.path.exists(latest_path):
    print("Resuming from checkpoint...")
    checkpoint = torch.load(latest_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    best_loss = checkpoint.get('best_loss', float('inf'))
    print(f"Resumed from epoch {start_epoch} with best loss {best_loss:.4f}")

# Training loop
for epoch in range(start_epoch, EPOCHS):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} finished. Average Loss: {avg_loss:.4f}")

    # Save latest checkpoint
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
        'best_loss': best_loss
    }, latest_path)
    print(f"📦 Latest checkpoint saved at {latest_path}")

    # Save best model if improved
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), best_path)
        print(f"🏆 Best model updated and saved at {best_path} (Loss: {best_loss:.4f})")

Epoch 1: 100%|██████████| 6396/6396 [18:08<00:00,  5.88it/s, loss=0.344]


Epoch 1 finished. Average Loss: 0.5024
📦 Latest checkpoint saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/latest.pt
🏆 Best model updated and saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/best.pt (Loss: 0.5024)


Epoch 2: 100%|██████████| 6396/6396 [18:13<00:00,  5.85it/s, loss=0.071]


Epoch 2 finished. Average Loss: 0.3510
📦 Latest checkpoint saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/latest.pt
🏆 Best model updated and saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/best.pt (Loss: 0.3510)


Epoch 3: 100%|██████████| 6396/6396 [18:13<00:00,  5.85it/s, loss=0.108]


...

Epoch 8 finished. Average Loss: 0.0529
📦 Latest checkpoint saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/latest.pt
🏆 Best model updated and saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/best.pt (Loss: 0.0529)


Epoch 9: 100%|██████████| 6396/6396 [18:13<00:00,  5.85it/s, loss=0.00122]


Epoch 9 finished. Average Loss: 0.0442
📦 Latest checkpoint saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/latest.pt
🏆 Best model updated and saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/best.pt (Loss: 0.0442)


Epoch 10: 100%|██████████| 6396/6396 [18:13<00:00,  5.85it/s, loss=0.0177]


Epoch 10 finished. Average Loss: 0.0388
📦 Latest checkpoint saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/latest.pt
🏆 Best model updated and saved at /content/drive/MyDrive/Web_Mining_DistilBert/checkpoints/best.pt (Loss: 0.0388)


In [7]:
from sklearn.metrics import accuracy_score, classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Accuracy:", accuracy_score(all_labels, all_preds))
print(classification_report(all_labels, all_preds))


Accuracy: 0.8554117968963765
              precision    recall  f1-score   support

           0       0.85      0.87      0.86      8527
           1       0.81      0.78      0.80      8528
           2       0.90      0.92      0.91      8528

    accuracy                           0.86     25583
   macro avg       0.85      0.86      0.85     25583
weighted avg       0.85      0.86      0.85     25583



In [ ]:
# Resume from a checkpoint
checkpoint_path = "bert_epoch_2.pt"  # replace with your file

checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']
